In [2]:
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)
%cd /content/gdrive/MyDrive/anirec

Mounted at /content/gdrive
/content/gdrive/MyDrive/anirec


In [3]:
!pip install -q tensorflow-recommenders
!pip install -q --upgrade tensorflow-datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.2/96.2 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.1/5.1 MB 23.1 MB/s eta 0:00:00


In [4]:
from typing import Dict, Text

import numpy as np
import pandas as pd
import tensorflow as tf

import tensorflow_datasets as tfds
import tensorflow_recommenders as tfrs

In [9]:
animes=pd.read_csv("processed_animes.csv")
ratings=pd.read_csv("processed_ratings.csv")

In [13]:
#can work here by using names instead of anime_id

animes = tf.data.Dataset.from_tensor_slices({
    'Name': animes['Name'].values,
  })

ratings = tf.data.Dataset.from_tensor_slices({
    'Name': ratings['Name'].values,
    'user_id': ratings['user_id'].values
})



In [14]:
user_ids_vocabulary = tf.keras.layers.StringLookup(mask_token=None)
user_ids_vocabulary.adapt(ratings.map(lambda x: x["user_id"]))

In [15]:
animes = animes.map(lambda x: x["Name"])

In [16]:
animes_vocabulary = tf.keras.layers.StringLookup(mask_token=None)
animes_vocabulary.adapt(animes)

In [17]:
class AniRecModel(tfrs.Model):

  def __init__(
      self,
      user_model: tf.keras.Model,
      anime_model: tf.keras.Model,
      task: tfrs.tasks.Retrieval):
    super().__init__()

    self.user_model = user_model
    self.anime_model = anime_model

    self.task = task

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:

    user_embeddings = self.user_model(features["user_id"])
    anime_embeddings = self.anime_model(features["Name"])

    return self.task(user_embeddings, anime_embeddings)

In [18]:
user_model = tf.keras.Sequential([
    user_ids_vocabulary,
    tf.keras.layers.Embedding(user_ids_vocabulary.vocab_size(), 64)
])
anime_model = tf.keras.Sequential([
    animes_vocabulary,
    tf.keras.layers.Embedding(animes_vocabulary.vocab_size(), 64)
])

task = tfrs.tasks.Retrieval(metrics=tfrs.metrics.FactorizedTopK(
    animes.batch(128).map(anime_model)
  )
)

In [19]:
model = AniRecModel(user_model, anime_model, task)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.5))

model.fit(ratings.batch(4096), epochs=50)

index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
index.index_from_dataset(
    animes.batch(100).map(lambda title: (title, model.anime_model(title))))

_, titles = index(np.array(["42"]))
print(f"Top 3 recommendations for user 42: {titles[0, :3]}")

Epoch 1/50
314/314 [==============================] - 153s 482ms/step - factorized_top_k/top_1_categorical_accuracy: 1.6584e-04 - factorized_top_k/top_5_categorical_accuracy: 0.0036 - factorized_top_k/top_10_categorical_accuracy: 0.0079 - factorized_top_k/top_50_categorical_accuracy: 0.0397 - factorized_top_k/top_100_categorical_accuracy: 0.0773 - loss: 34059.8225 - regularization_loss: 0.0000e+00 - total_loss: 34059.8225
Epoch 2/50
314/314 [==============================] - 127s 405ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0139 - factorized_top_k/top_5_categorical_accuracy: 0.0756 - factorized_top_k/top_10_categorical_accuracy: 0.1185 - factorized_top_k/top_50_categorical_accuracy: 0.2787 - factorized_top_k/top_100_categorical_accuracy: 0.3846 - loss: 30778.5929 - regularization_loss: 0.0000e+00 - total_loss: 30778.5929
Epoch 3/50
314/314 [==============================] - 131s 419ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0237 - factorized_top_k/top_5_cat

In [ ]:
import tempfile
import os

path="anirec_model/"

tf.saved_model.save(index, path)

In [23]:
loaded = tf.saved_model.load(path)
scores, titles = loaded(["5"])
print(f"Recommendations: {titles[0][:10]}")

Recommendations: [b'Zoids: New Century/Zero' b'Gundam Build Fighters Try'
 b'Sword Art Online Alternative: Gun Gale Online' b'Mobile Suit Gundam 00'
 b'Cowboy Bebop Session XX: Mish-Mash Blues' b'Digimon Season 3: Tamers'
 b'Digimon Season 1: Digital Monsters'
 b'Digimon Season 2: Digital Monsters' b'Air Gear' b'Shaman King']


In [11]:
import tensorflow as tf

def detach_recommendations(path, user_item_dict):

  user_ids=list(user_item_dict.keys())

  loaded = tf.saved_model.load(path)
  user_recommendations = {}

  for user_id in user_ids:
    scores, titles = loaded([str(user_id)])
    recommendations = titles[0][:10].numpy().tolist()
    recommendations = [recommendation.decode("utf-8") for recommendation in recommendations]
    user_recommendations[user_id] = recommendations

  return user_recommendations